In [ ]:
# BERTScoreとその依存関係をインストール
!pip install bert-score torch transformers

In [ ]:
import pandas as pd
import os
from typing import Union, List

# BERTScoreのインポート
from bert_score import score

# pandasの小数点以下の桁数を3に設定する
pd.options.display.precision = 3

# --- 1. BERTScore計算のためのヘルパー関数 ---

def compute_bert_score(
    predictions: List[str], references: List[str]
) -> dict[str, dict[str, float]]:
    """
    BERTScoreを算出
    """
    # 言語モデルの指定（日本語の評価に適したモデルを使用）
    # 'cl-tohoku/bert-base-japanese-whole-word-masking' などが一般的
    model_type = "bert-base-multilingual-cased"

    # BERTScoreの計算
    # P: Precision, R: Recall, F1: F-measure (F1スコア)
    # BERTScoreは単語の意味的な類似度（埋め込みベクトル）を評価します。
    P, R, F1 = score(
        predictions,
        references,
        lang="ja", # 言語指定
        model_type=model_type,
        verbose=True # 処理状況を表示
    )

    # 結果の平均を辞書形式で返す
    avg_scores = {
        "bertscore_precision": P.mean().item(),
        "bertscore_recall": R.mean().item(),
        "bertscore_f1": F1.mean().item(),
    }

    # ROUGEの結果と同じ形式で、Precision, Recall, F-measureを返す
    return {
        "bertscore": {
            "fmeasure": avg_scores["bertscore_f1"],
            "precision": avg_scores["bertscore_precision"],
            "recall": avg_scores["bertscore_recall"],
        }
    }

# --- 2. ファイル/文字列 入力処理関数 ---

def load_data(input_data: Union[str, List[str]]) -> List[str]:
    """
    入力データ（ファイルパスまたは文字列のリスト）を読み込み、元の文のリストを返す
    """
    data_list = []

    if isinstance(input_data, str):
        # 文字列がファイルパスとして存在する場合
        if os.path.exists(input_data):
            print(f"✅ ファイル '{input_data}' からデータを読み込みます。")
            with open(input_data, 'r', encoding='utf-8') as f:
                data_list = [line.strip() for line in f if line.strip()]
        else:
            # 存在しない場合は、その文字列を単一のデータとして扱う
            print(f"💡 入力文字列を単一のデータとして扱います。")
            data_list = [input_data]

    elif isinstance(input_data, list):
        # リストが与えられた場合
        print(f"💡 リスト入力されたデータを扱います。")
        data_list = input_data

    # BERTScoreでは分かち書きは不要なので、そのまま返します。
    return data_list

def compute_bert_score_from_input(
    predictions: Union[str, List[str]],
    references: Union[str, List[str]]
) -> pd.DataFrame:
    """
    生成文と正解文の入力形式（ファイルパスまたはリスト）を処理し、BERTScoreを算出する
    """

    # データ読み込み（分かち書きなし）
    ref_list = load_data(references)
    pred_list = load_data(predictions)

    if len(ref_list) != len(pred_list):
        raise ValueError(
            f"参照文と生成文の数が一致しません。参照文: {len(ref_list)}、生成文: {len(pred_list)}"
        )

    # BERTScoreの計算
    bert_results = compute_bert_score(pred_list, ref_list)

    # 結果をPandas DataFrameとして整形して表示
    df = pd.DataFrame.from_dict(bert_results, orient="index")
    return df

# --- 3. 実行例 ---

# 1. 直接文字列（単一）を与える例
print("--- 実行例 1: 直接文字列（単一）を与える場合 ---")
single_ref = "「春」をキーワードに星加アナウンサーが松山・石手を歩く。春の河川敷でビーチバレーの特訓をする若者やこの春入学を控える新一年生、自宅の壁一面にレトロ看板を「貼る」夫婦に遭遇。食欲の春！一日７５０本売れる食パンに、老舗うどん店の「メロンうどん」とは？（写真は一場面）"
single_pred = "去年９月にオープンした「パン屋アシヤ」 商品はこの食パンのみ！食パン一本で勝負する専門店です。 一日７００本以上売れるんですから、オーブンは大忙し！ プルプルもちもちのプレミアム食パンです。 二人が追い求めたうどんは ある食べ物に似た食感に仕上がります。"
df_single = compute_bert_score_from_input(single_pred, single_ref)
print("\n✅ スコア結果 (単一文字列):")
display(df_single)
print("-" * 50)


# 2. 複数の文をリストとして与える例
print("--- 実行例 2: リスト（複数）を与える場合 ---")
multi_refs = ["私は昨日ケーキを食べた。", "明日の天気は晴れです。"]
multi_preds = ["昨日、私はケーキを食べる。", "明日の天気が晴れだ。"]
df_multi = compute_bert_score_from_input(multi_preds, multi_refs)
print("\n✅ スコア結果 (リスト):")
display(df_multi)
print("-" * 50)


# 3. ファイルパスとして与える例 (ファイルが存在しない場合は動作しません)
# 実行する場合は、Colabに 'ref.txt' と 'pred.txt' をアップロードしてから、コメントアウトを外してください。

# print("--- 実行例 3: ファイルパスとして与える場合 ---")
# file_ref = "ref.txt"
# file_pred = "pred.txt"
# if os.path.exists(file_ref) and os.path.exists(file_pred):
#     df_file = compute_bert_score_from_input(file_pred, file_ref)
#     print("\n✅ スコア結果 (ファイル):")
#     display(df_file)
# else:
#     print(f"⚠️ ファイル '{file_ref}' または '{file_pred}' が見つかりませんでした。")
#     print("Colabにファイルをアップロードしてから試してください。")